# Importing the necessary Libraries

In [ ]:
import numpy as np
import tensorflow as tf
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.models import Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# ***In this Project, we are using the TensorFlow Framework***.

# The provided code defines a class named **traffic_sign_classifier** that encapsulates functionalities related to traffic sign classification using **Convolutional Neural Networks (CNNs).**


1. ***Initialization***: The constructor __init__ loads data from a pickle file (data0.pickle) containing training, validation, and test datasets for images of traffic signs. It organizes the data into attributes like x_train, y_train, x_valid, y_valid, x_test, y_test, and labels.
2.  ***Data Exploration***: The method print_shape prints the shape of different datasets, providing insight into the dimensions of the image data.
3. ***Visualization***: The method plot_grid plots a grid of images with their corresponding labels. It randomly selects images from the training dataset and displays them in a grid format.
4. ***Preprocessing***: The method gray_normalization performs data preprocessing steps. It shuffles the training dataset, converts the images to grayscale, and then normalizes the pixel values to fall within the range [-1, 1].
5. ***Model Definition***: The method build_CNN_model constructs a CNN model using the Keras Sequential API. It defines layers for convolution, max-pooling, dropout, flattening, and dense (fully connected) layers, ending with a softmax layer for multi-class classification.
6. ***Model Management***: The methods save_model and load_model_data handle saving and loading trained models using Keras.
7. ***Model Training***: The method compile_and_train compiles the CNN model with specified optimizer, loss function, and metrics. It then trains the model on the preprocessed training data for a specified number of epochs while validating on a separate validation dataset.
8. ***Model Training***: The method compile_and_train compiles the CNN model with specified optimizer, loss function, and metrics. It then trains the model on the preprocessed training data for a specified number of epochs while validating on a separate validation dataset.
9. ***Training Visualization***: The method plot_metrics visualizes the training and validation accuracy and loss over epochs using matplotlib.
10. ***Model Evaluation***: The method evaluate_model evaluates the trained model's performance on the test dataset and prints the evaluation metrics (typically accuracy and loss).
11. ***Testing***: The method test_plot performs inference on a subset of the test dataset and plots the original images alongside their predicted classes. This helps in visually assessing the model's performance on unseen data.

 # **Overall, this class provides a comprehensive framework for traffic sign classification, covering data loading, preprocessing, model construction, training, evaluation, and visualization.**





In [ ]:
class traffic_sign_classifier():

    def __init__(self):

        with open('data0.pickle', 'rb') as file1:
            data = pickle.load(file1, encoding='latin1')

        #load data--------------------------------
        self.x_train = data['x_train'].transpose(0,2,3,1)
        self.y_train = data['y_train']
        self.x_valid = data['x_validation'].transpose(0,2,3,1)
        self.y_valid = data['y_validation']
        self.x_test = data['x_test'].transpose(0,2,3,1)
        self.y_test = data['y_test']
        self.labels = data['labels']
    #------------------------------------------

    def print_shape(self):
        print('x_train: {}'.format(self.x_train.shape))
        print('y_train: {}'.format(self.y_train.shape))
        print('x_valid: {}'.format(self.x_valid.shape))
        print('y_valid: {}'.format(self.y_valid.shape))
        print('x_test: {}'.format(self.x_test.shape))
        print('y_test: {}'.format(self.y_test.shape))

    def plot_grid(self, l_grid, w_grid):

        #plotting images and labels in a grid of 5 x 5:
        l_grid = 5
        w_grid = 5
        _, axes = plt.subplots(l_grid, w_grid, figsize = (10,10))
        axes = axes.ravel()
        for i in np.arange(0, l_grid * w_grid):
            j = np.random.randint(i, len(self.x_train))
            axes[i].imshow(self.x_train[j])
            axes[i].set_title(self.labels[self.y_train[j]], fontsize = 10)
            axes[i].axis('off')
        plt.subplots_adjust(hspace=0.5)
        plt.show()

    def gray_normalization(self):
        #shuffling datasets to prevent the NN to learn any possible sequence:
        self.x_train, self.y_train = shuffle(self.x_train, self.y_train)
        #grayscale_conversion:
        self.x_train_gray_norm = np.sum(self.x_train/3, axis = 3, keepdims=True)
        self.x_valid_gray_norm = np.sum(self.x_valid/3, axis = 3, keepdims=True)
        self.x_test_gray_norm = np.sum(self.x_test/3, axis = 3, keepdims=True)

        #normalization:-
        self.x_train_gray_norm = (self.x_train_gray_norm - 128) / 128
        self.x_valid_gray_norm = (self.x_valid_gray_norm - 128) / 128
        self.x_test_gray_norm = (self.x_test_gray_norm - 128) / 128

        #print('x_train_gray_norm: {}'.format(self.x_train_gray_norm.shape))
        #print('x_valid_gray_norm: {}'.format(self.x_valid_gray_norm.shape))
        #print('x_test_gray_norm: {}'.format(self.x_test_gray_norm.shape))

    def build_CNN_model(self, dropout = 0.2):
        self.CNN = Sequential([
            Conv2D(16, (3,3), padding = 'same', activation='relu', input_shape = (32,32,1)),
            MaxPooling2D((2,2)),
            Conv2D(32, (3,3), padding = 'same', activation='relu'),
            MaxPooling2D(2,2),
            Dropout(dropout),
            Conv2D(64, (3,3), activation='relu'),
            MaxPooling2D((2,2)),
            Flatten(),
            Dense(128, activation='relu'),
            Dense(86,activation='relu'),
            Dense(43, activation='softmax')
        ])

    def save_model(self, save_path):
        self.CNN.save(save_path)

    def load_model_data(self, load_path):
        self.CNN = load_model(load_path)

    def compile_and_train(self, lr=0.001):
        self.CNN.compile(optimizer = Adam(learning_rate = lr), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
        self.history = self.CNN.fit(self.x_train_gray_norm, self.y_train, batch_size = 500, epochs = 100,
                                    verbose = 3, validation_data = (self.x_valid_gray_norm, self.y_valid))
        self.score = self.CNN.evaluate(self.x_test_gray_norm, self.y_test)

    def plot_metrics(self):

        accuracy = self.history.history['accuracy']
        val_accuracy = self.history.history['val_accuracy']
        loss = self.history.history['loss']
        val_loss = self.history.history['val_loss']


        epochs = range(len(accuracy))
        plt.figure(1)
        plt.plot(epochs, accuracy, 'r-', label = 'Training accuracy')
        plt.plot(epochs, val_accuracy, 'b-', label = 'Validation accuracy')
        plt.title('Training and Validation Accuracy')
        plt.show()

        plt.figure(2)
        plt.plot(epochs, loss, 'r-', label = 'Training loss')
        plt.plot(epochs, val_loss, 'b-', label = 'Validation loss')
        plt.title('Training and Validation loss')
        plt.show()

    def evaluate_model(self):
        score = self.CNN.evaluate(self.x_test_gray_norm, self.y_test, batch_size = 500)
        print(score)

    def test_plot(self):
        predicted_classes = self.CNN.predict_classes(self.x_test_gray_norm)
        l_grid = 2
        w_grid = 2
        _, axes = plt.subplots(l_grid, w_grid, figsize = (12,12))
        axes = axes.ravel()
        for i in np.arange(0, l_grid * w_grid):
            #j = np.random.randint(i, len(self.x_test))
            axes[i].imshow(self.x_test[i])
            # axes[i].set_title(self.labels[self.y_test[j]], fontsize = 8)
            print('Predicted_class: {} -- Label: {}'.format(predicted_classes[i], self.y_test[i]))
            axes[i].axis('off')
        plt.subplots_adjust(hspace=0.5)
        plt.show()

# Here, you've created an instance of the **traffic_sign_classifier** class named tfsc and called the print_shape method to display the shapes of different datasets.

In [ ]:
import matplotlib.pyplot as plt #  Imports the matplotlib library for data visualization and assigns it the alias plt.
tfsc = traffic_sign_classifier() # Creates an instance of the traffic_sign_classifier class and assigns it to the variable tfsc
import numpy as np
tfsc.print_shape() # Calls the print_shape method of the tfsc object, which prints the shapes of different datasets.

# The remaining lines of code are commented out and are not executed.
# ***These method calls are essential for various stages of the machine learning workflow, including data preprocessing, model construction, training, evaluation, and visualization of results. Uncommenting and executing these lines would enable you to perform these tasks and analyze the performance of your traffic sign classifier model.***

In [ ]:
#tfsc.plot_grid(5,5)
# This method call likely plots a grid of images along with their corresponding labels. The parameters (5,5) indicate the dimensions of the grid.

#tfsc.gray_normalization()
# performs grayscale conversion and normalization on the datasets.
# It shuffles the datasets to prevent the neural network from learning any sequence and then normalizes the datasets.

#tfsc.build_CNN_model(dropout=0.4)
# constructs a Convolutional Neural Network (CNN) model.
#The parameter dropout=0.4 indicates the dropout rate, which is a regularization technique used to prevent overfitting.

#accuracy =
#tfsc.CNN.summary()
#prints a summary of the CNN model, including information about the layers, parameters, and output shapes.

#tfsc.load_model_data(load_path = 'model_dropout_4.h5')
#loads a pre-trained model from the specified file path ('model_dropout_4.h5').

#tfsc.compile_and_train()
# compiles and trains the CNN model using the training dataset.
#It specifies the optimizer, loss function, and metrics for training, as well as the number of epochs and batch size.

#tfsc.test_plot()
#plots a grid of images from the test dataset along with their predicted classes.
#It helps visualize the performance of the trained model on unseen data.

#tfsc.save_model(save_path = 'model_dropout_4.h5')
# saves the trained model to the specified file path ('model_dropout_4.h5').

#tfsc.plot_metrics()
# plots the training and validation metrics (e.g., accuracy and loss) over epochs to visualize the training progress and model performance.

#tfsc.evaluate_model()
# evaluates the performance of the trained model on the test dataset and prints the evaluation score (e.g., accuracy).